In [24]:
from keras.layers import Bidirectional, Dense, Embedding, Concatenate, Dot, Flatten, Input, Lambda, LSTM, merge, GlobalAveragePooling1D, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD, RMSprop
from keras import objectives
from keras import backend as K
from keras.models import Model
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import csv
import os

In [2]:
batch_size = 256
max_len = 10
emb_dim = 50
NB_WORDS = 50000
latent_dim = 50
intermediate_dim = 96
epsilon_std = 1.0
num_sampled=500
act = ELU()

In [3]:
%%time
import pandas as pd

TRAIN_DATA_FILE = '/data/chzho/deepqts/train_data/unifiedclick/join_oneyearsample_2B_training_all_top10'
num_read_row = 1000000
df = pd.read_csv(TRAIN_DATA_FILE, sep="\t", usecols=[0,1,3], names=['label', 'q', 'd'], header=None , error_bad_lines=False, nrows=num_read_row)
df = df.dropna()

TEST_DATA_FILE = '/data/chzho/deepqts/test_data/uhrs/unified/uhrs_do_10'
df_qd = pd.read_csv(TEST_DATA_FILE, sep="\t", usecols=[0,1,3,5], names=['label', 'q', 'd', 'market'], header=None , error_bad_lines=False)
df_qd = df_qd.dropna()
df_qd = df_qd[df_qd.market == "en-US"]

TEST_DATA_FILE = '/data/chzho/deepqts/test_data/julyflower/julyflower_original.tsv'
df_qq = pd.read_csv(TEST_DATA_FILE, sep="\t", names=['q', 'd', 'label'], header=None , error_bad_lines=False)
df_qq = df_qq.dropna()

CPU times: user 18.9 s, sys: 2.68 s, total: 21.6 s
Wall time: 41.4 s


In [4]:
TRAIN_DATA_FILE = '/data/t-mipha/data/agi_encoder/v4/universal/CLICKED_QQ_EN_universal_train_1M.txt'
df_qq_click = pd.read_csv(TRAIN_DATA_FILE, usecols=[0,1], names=["q", "sim_q"], sep="\t", header=None, error_bad_lines=False)
df_qq_click = df_qq_click.dropna()

In [6]:
q1_qq_click_raw = df_qq_click.q.tolist()
q2_qq_click_raw = [i.split("<sep>")[0] for i in df_qq_click.sim_q.tolist()]
y_qq_click_train = np.ones(len(df_qq_click))

In [7]:
texts = q1_qq_click_raw + q2_qq_click_raw + df_qd.q.tolist() + df_qd.d.tolist() + df_qq.q.tolist() + df_qq.d.tolist() + df.q.tolist() + df.d.tolist()


In [4]:
%%time

tokenizer = Tokenizer(NB_WORDS)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index #the dict values start from 1 so this is fine with zeropadding
index2word = {v: k for k, v in word_index.items()}
print('Found %s unique tokens' % len(word_index))
NB_WORDS = (min(tokenizer.num_words, len(word_index)) + 1 ) #+1 for zero padding
print('Number of Vocab: %d' % NB_WORDS)



Found 210261 unique tokens
Number of Vocab: 200001
CPU times: user 14.5 s, sys: 172 ms, total: 14.7 s
Wall time: 14.6 s


In [6]:
uns_q = pad_sequences(tokenizer.texts_to_sequences(df.q.tolist()), maxlen=max_len)
uns_d = pad_sequences(tokenizer.texts_to_sequences(df.d.tolist()), maxlen=max_len)

In [57]:



def create_lstm_vae(input_dim, 
    timesteps, 
    batch_size, 
    intermediate_dim, 
    latent_dim,
    epsilon_std=1.):

    """
    Creates an LSTM Variational Autoencoder (VAE). Returns VAE, Encoder, Generator. 
    # Arguments
        input_dim: int.
        timesteps: int, input timestep dimension.
        batch_size: int.
        intermediate_dim: int, output shape of LSTM. 
        latent_dim: int, latent z-layer shape. 
        epsilon_std: float, z-layer sigma.
    # References
        - [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
        - [Generating sentences from a continuous space](https://arxiv.org/abs/1511.06349)
    """
    x = Input(shape=(timesteps, input_dim,))

    # LSTM encoding
    h = LSTM(intermediate_dim)(x)

    # VAE Z layer
    z_mean = Dense(latent_dim)(h)
    z_log_sigma = Dense(latent_dim)(h)
    
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim),
                                  mean=0., stddev=epsilon_std)
        return z_mean + z_log_sigma * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    # so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
    
    # decoded LSTM layer
    decoder_h = LSTM(intermediate_dim, return_sequences=True)
    decoder_mean = LSTM(input_dim, return_sequences=True)

    h_decoded = RepeatVector(timesteps)(z)
    h_decoded = decoder_h(h_decoded)

    # decoded layer
    x_decoded_mean = decoder_mean(h_decoded)
    
    # end-to-end autoencoder
    vae = Model(x, x_decoded_mean)

    # encoder, from inputs to latent space
    encoder = Model(x, [z_mean, z_log_sigma, z])

    # generator, from latent space to reconstructed inputs
    decoder_input = Input(shape=(latent_dim,))

    _h_decoded = RepeatVector(timesteps)(decoder_input)
    _h_decoded = decoder_h(_h_decoded)

    _x_decoded_mean = decoder_mean(_h_decoded)
    generator = Model(decoder_input, _x_decoded_mean)
    
    def vae_loss(x, x_decoded_mean):
        xent_loss = objectives.mse(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
        loss = xent_loss + kl_loss
        return loss

    vae.compile(optimizer='rmsprop', loss=vae_loss)
    
    return vae, encoder, generator
vae, encoder, decoder = create_lstm_vae(emb_dim, max_len, batch_size, intermediate_dim, latent_dim)

In [23]:
def get_w2v():
    GLOVE_EMBEDDING = '/home/t-jamano/data/glove/glove.6B.50d.txt'
    embeddings_index = {}
    f = open(GLOVE_EMBEDDING, encoding='utf8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))


    glove_embedding_matrix = np.zeros((NB_WORDS, emb_dim))
    for word, i in word_index.items():
        if i < NB_WORDS:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be the word embedding of 'unk'.
                glove_embedding_matrix[i] = embedding_vector
            else:
                glove_embedding_matrix[i] = embeddings_index.get('unk')
    print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))
    
    w2v = Sequential()
    w2v.add(Embedding(NB_WORDS, emb_dim, input_length=max_len, weights=[glove_embedding_matrix]))
    
    return w2v

In [24]:
w2v = get_w2v()

Found 400000 word vectors.
Null word embeddings: 1


In [25]:
uns_q.shape

(100000, 5)

In [28]:
uns_q = w2v.predict(uns_q)
uns_d = w2v.predict(uns_d)

In [29]:
vae.fit(uns_q, uns_q,  verbose=2, batch_size=batch_size, epochs=20)

Epoch 1/20
 - 19s - loss: 0.2456
Epoch 2/20
 - 17s - loss: 0.2148
Epoch 3/20
 - 17s - loss: 0.2012
Epoch 4/20
 - 17s - loss: 0.1913
Epoch 5/20
 - 17s - loss: 0.1841
Epoch 6/20
 - 17s - loss: 0.1786
Epoch 7/20
 - 17s - loss: 0.1739
Epoch 8/20
 - 17s - loss: 0.1702
Epoch 9/20
 - 17s - loss: 0.1672
Epoch 10/20
 - 17s - loss: 0.1644
Epoch 11/20
 - 17s - loss: 0.1620
Epoch 12/20
 - 17s - loss: 0.1597
Epoch 13/20
 - 17s - loss: 0.1577
Epoch 14/20
 - 17s - loss: 0.1559
Epoch 15/20
 - 17s - loss: 0.1542
Epoch 16/20
 - 17s - loss: 0.1527
Epoch 17/20
 - 17s - loss: 0.1513
Epoch 18/20
 - 17s - loss: 0.1500
Epoch 19/20
 - 17s - loss: 0.1489
Epoch 20/20
 - 17s - loss: 0.1477


In [30]:
vae.fit(uns_d, uns_d,  verbose=2, batch_size=batch_size, epochs=20)

Epoch 1/20
 - 17s - loss: 0.2372
Epoch 2/20
 - 17s - loss: 0.2325
Epoch 3/20
 - 17s - loss: 0.2289
Epoch 4/20
 - 17s - loss: 0.2258
Epoch 5/20
 - 17s - loss: 0.2229
Epoch 6/20
 - 17s - loss: 0.2201
Epoch 7/20
 - 17s - loss: 0.2175
Epoch 8/20
 - 17s - loss: 0.2151
Epoch 9/20
 - 17s - loss: 0.2130
Epoch 10/20
 - 17s - loss: 0.2109
Epoch 11/20
 - 17s - loss: 0.2089
Epoch 12/20
 - 17s - loss: 0.2070
Epoch 13/20
 - 17s - loss: 0.2054
Epoch 14/20
 - 17s - loss: 0.2039
Epoch 15/20
 - 17s - loss: 0.2025
Epoch 16/20
 - 17s - loss: 0.2012
Epoch 17/20
 - 17s - loss: 0.1999
Epoch 18/20
 - 17s - loss: 0.1987
Epoch 19/20
 - 17s - loss: 0.1974
Epoch 20/20
 - 17s - loss: 0.1961


In [33]:
q_qq = pad_sequences(tokenizer.texts_to_sequences(df_qq.q.tolist()), maxlen=max_len)
d_qq = pad_sequences(tokenizer.texts_to_sequences(df_qq.d.tolist()), maxlen=max_len)
# q_qd = pad_sequences(tokenizer.texts_to_sequences(df_qd.q.tolist()), maxlen=max_len)
# d_qd = pad_sequences(tokenizer.texts_to_sequences(df_qd.d.tolist()), maxlen=max_len)


# first encode with W2V
q_qq = w2v.predict(q_qq)
d_qq = w2v.predict(d_qq)
# q_qd = w2v.predict(q_qd)
# d_qd = w2v.predict(d_qd)

In [15]:

class W2V_MLP():
    def __init__(self, max_len, input_dim):

        que_input = Input(shape=(max_len, input_dim,))
        doc_input = Input(shape=(max_len, input_dim,))
        
        x = GlobalAveragePooling1D()(que_input)
        y = GlobalAveragePooling1D()(doc_input)

        concat = merge([x, y], mode="concat")


        pred = Dense(1, activation='sigmoid')(concat)

        self.model = Model(input=[que_input, doc_input], output=pred)
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

class AVGPolling():
    def __init__(self, max_len, emb_dim):
        x_input = Input(shape=(max_len, emb_dim,))
        avg = GlobalAveragePooling1D()(x_input)
        self.model = Model(input=x_input, output=avg)


class LSTM():
    def __init__(self, max_len, input_dim):
        
        que_input = Input(shape=(max_len, input_dim,))
        doc_input = Input(shape=(max_len, input_dim,))
        
        emb = Embedding()
        
        x = GlobalAveragePooling1D()(que_input)
        y = GlobalAveragePooling1D()(doc_input)

        concat = merge([x, y], mode="concat")


        pred = Dense(1, activation='sigmoid')(concat)

        self.model = Model(input=[que_input, doc_input], output=pred)
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [36]:
# build a model to project sentences on the latent space# build 
# encoder = Model(x, z)
enc_q_qq = encoder.predict(q_qq)
enc_d_qq = encoder.predict(d_qq)

In [48]:
mlp = MLP(latent_dim)
mlp.model.fit([enc_q_qq, enc_d_qq], df_qq.label.values, verbose=2, batch_size=8, validation_split=0.5, epochs=20)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/t-jamano/.local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 213 samples, validate on 213 samples
Epoch 1/20
 - 1s - loss: 0.6945 - acc: 0.4977 - val_loss: 0.6856 - val_acc: 0.6197
Epoch 2/20
 - 0s - loss: 0.6914 - acc: 0.5258 - val_loss: 0.6841 - val_acc: 0.6150
Epoch 3/20
 - 0s - loss: 0.6885 - acc: 0.5587 - val_loss: 0.6828 - val_acc: 0.6291
Epoch 4/20
 - 0s - loss: 0.6860 - acc: 0.5446 - val_loss: 0.6812 - val_acc: 0.6385
Epoch 5/20
 - 0s - loss: 0.6832 - acc: 0.5587 - val_loss: 0.6800 - val_acc: 0.6479
Epoch 6/20
 - 0s - loss: 0.6808 - acc: 0.5587 - val_loss: 0.6789 - val_acc: 0.6197
Epoch 7/20
 - 0s - loss: 0.6784 - acc: 0.5822 - val_loss: 0.6776 - val_acc: 0.6056
Epoch 8/20
 - 0s - loss: 0.6760 - acc: 0.6103 - val_loss: 0.6766 - val_acc: 0.6244
Epoch 9/20
 - 0s - loss: 0.6738 - acc: 0.6103 - val_loss: 0.6757 - val_acc: 0.6197
Epoch 10/20
 - 0s - loss: 0.6716 - acc: 0.6244 - val_loss: 0.6747 - val_acc: 0.6150
Epoch 11/20
 - 0s - loss: 0.6696 - acc: 0.6150 - val_loss: 0.6738 - val_acc: 0.6103
Epoch 12/20
 - 0s - loss: 0.6675 - acc:

In [50]:
q_qq.shape

(426, 5, 50)

In [106]:
def split_data(q, d, df, size, axis=1):
    concat = np.concatenate((q, d), axis=axis)
    label = df.label.values
    return train_test_split(concat, label, test_size=size, random_state=42)
    

x_train, x_test, y_train, y_test = split_data(enc_q_qq, enc_d_qq, df_qq, 0.5)

In [87]:
pred = mlp.model.predict([x_test[:, :latent_dim], x_test[:, latent_dim:]]).flatten()

In [88]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.72005323868677906

In [82]:
w2v_mlp = W2V_MLP(max_len, emb_dim)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/t-jamano/.local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [100]:
w2v_mlp.model.fit([q_qq, d_qq], df_qq.label.values, verbose=2, batch_size=8, validation_split=0.5, epochs=20)

Train on 213 samples, validate on 213 samples
Epoch 1/20
 - 1s - loss: 0.7041 - acc: 0.5634 - val_loss: 0.7223 - val_acc: 0.5164
Epoch 2/20
 - 0s - loss: 0.6918 - acc: 0.5728 - val_loss: 0.7147 - val_acc: 0.5211
Epoch 3/20
 - 0s - loss: 0.6827 - acc: 0.5822 - val_loss: 0.7099 - val_acc: 0.5211
Epoch 4/20
 - 0s - loss: 0.6744 - acc: 0.5728 - val_loss: 0.7058 - val_acc: 0.5117
Epoch 5/20
 - 0s - loss: 0.6672 - acc: 0.5775 - val_loss: 0.7020 - val_acc: 0.5305
Epoch 6/20
 - 0s - loss: 0.6602 - acc: 0.5869 - val_loss: 0.7001 - val_acc: 0.5399
Epoch 7/20
 - 0s - loss: 0.6542 - acc: 0.5869 - val_loss: 0.6975 - val_acc: 0.5446
Epoch 8/20
 - 0s - loss: 0.6482 - acc: 0.6150 - val_loss: 0.6949 - val_acc: 0.5493
Epoch 9/20
 - 0s - loss: 0.6432 - acc: 0.6291 - val_loss: 0.6932 - val_acc: 0.5446
Epoch 10/20
 - 0s - loss: 0.6377 - acc: 0.6620 - val_loss: 0.6918 - val_acc: 0.5493
Epoch 11/20
 - 0s - loss: 0.6337 - acc: 0.6761 - val_loss: 0.6910 - val_acc: 0.5540
Epoch 12/20
 - 0s - loss: 0.6291 - acc:

In [101]:
x_train, x_test, y_train, y_test = split_data(q_qq, d_qq, df_qq, 0.5, 2)

pred = w2v_mlp.model.predict([x_test[:, :, :latent_dim], x_test[:, :, latent_dim:]]).flatten()

In [12]:
def auc(y_test, pred):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=1)
    return metrics.auc(fpr, tpr)

In [117]:
avg = AVGPolling(max_len, emb_dim)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


In [11]:
def get_cosine_sim(x, y):
    tmp = []
    for i,j in zip(x,y):
        tmp.append(cosine_similarity(i.reshape(1, -1),j.reshape(1, -1)))
    return np.array(tmp).flatten()
        

In [133]:
vae_pred = matrix_cosine(enc_q_qq, enc_d_qq)
w2v_pred = matrix_cosine(avg.model.predict(q_qq), avg.model.predict(d_qq))

In [8]:
%%time
from l3wtransformer import L3wTransformer
l3wt = L3wTransformer()
l3wt.fit_on_texts(texts)

CPU times: user 2min 35s, sys: 64 ms, total: 2min 35s
Wall time: 2min 35s


In [9]:
%%time 
max_len = 10
# Query-Document Log
# q_train = pad_sequences(l3wt.texts_to_sequences(df.q.tolist()), maxlen=max_len)
# d_train = pad_sequences(l3wt.texts_to_sequences(df.d.tolist()), maxlen=max_len)

# Query-Document Testing data
# q_qd_test = pad_sequences(l3wt.texts_to_sequences(df_qd.q.tolist()), maxlen=max_len)
# d_qd_test = pad_sequences(l3wt.texts_to_sequences(df_qd.d.tolist()), maxlen=max_len)

# Query-Query Testing data
q1_qq_test = pad_sequences(l3wt.texts_to_sequences(df_qq.q.tolist()), maxlen=max_len)
q2_qq_test = pad_sequences(l3wt.texts_to_sequences(df_qq.d.tolist()), maxlen=max_len)

# Query-Query Click data from Minh
q1_qq_click_train = pad_sequences(l3wt.texts_to_sequences(q1_qq_click_raw), maxlen=max_len)
q2_qq_click_train = pad_sequences(l3wt.texts_to_sequences(q2_qq_click_raw), maxlen=max_len)

CPU times: user 1min 49s, sys: 1 s, total: 1min 50s
Wall time: 1min 49s


In [338]:
np.max([len(i) for i in l3wt.texts_to_sequences(df_qq.q.tolist())])
max_len = 10
# Query-Query data
q1_qq_test = pad_sequences(l3wt.texts_to_sequences(df_qq.q.tolist()), maxlen=max_len)
q2_qq_test = pad_sequences(l3wt.texts_to_sequences(df_qq.d.tolist()), maxlen=max_len)

In [46]:
class LSTM_Model():
    def __init__(self, max_len=10, emb_dim=100, nb_words=50000):

        q_input = Input(shape=(max_len,))
        d_input = Input(shape=(max_len,))
        
        emb = Embedding(nb_words, emb_dim, mask_zero=True)

        lstm = LSTM(256)

        self.q_embed = lstm(emb(q_input))
        self.d_embed = lstm(emb(d_input))

        concat = Concatenate()([self.q_embed, self.d_embed])

        pred = Dense(1, activation='sigmoid')(concat)

        self.model = Model(inputs=[q_input, d_input], outputs=pred)
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

lstm_model = LSTM_Model()

In [33]:
class MLP():
    def __init__(self, input_dim):

        que_input = Input(shape=(input_dim,))
        doc_input = Input(shape=(input_dim,))
        
        concat = Concatenate()([que_input, doc_input])

        pred = Dense(1, activation='sigmoid')(concat)

        self.model = Model(input=[que_input, doc_input], output=pred)
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Require Negative Sample, otherwise model only predicts 1

In [50]:
mlp_model = MLP(max_len)

q2_neg_qq_click_train = np.copy(q2_qq_click_train)
# np.random.shuffle(q2_neg_qq_click_train)
lstm_model = LSTM_Model()
# x_train = [np.concatenate((q1_qq_click_train, q1_qq_click_train)), np.concatenate((q2_qq_click_train, q2_neg_qq_click_train))] 
y_train = np.array([1] * len(df_qq_click) + [0] * len(df_qq_click))
for i in range(3):
    
#     np.random.shuffle(q2_neg_qq_click_train)
#     x_train = [np.concatenate((q1_qq_click_train, q1_qq_click_train)), np.concatenate((q2_qq_click_train, q2_neg_qq_click_train))] 

    lstm_model.model.fit(x_train, y_train, verbose=2, batch_size=256)

    pred = lstm_model.model.predict([q1_qq_test, q2_qq_test])
    print(auc(df_qq.label.values, pred))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/t-jamano/.local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/1
 - 401s - loss: 0.6932 - acc: 0.4997
0.509195223376
Epoch 1/1
 - 401s - loss: 0.6932 - acc: 0.4991
0.474715915408
Epoch 1/1
 - 403s - loss: 0.6932 - acc: 0.4986
0.513509306411


In [81]:
# y_test = [0 if i == "Bad" else 1 for i in df_qd.label.tolist()]

In [76]:
batch_size = 1000
max_len = 10
emb_dim = 50
NB_WORDS = 50000
latent_dim = 100
intermediate_dim = 256
epsilon_std = 1.0
num_sampled=500
act = ELU()
vae, encoder, decoder = create_lstm_vae(emb_dim, max_len, batch_size, intermediate_dim, latent_dim)

In [388]:
# http://alexadam.ca/ml/2017/05/05/keras-vae.html
class EMB_LSTM_VAE():
    def __init__(self, vocab_size=50000, max_length=10, latent_rep_size=50):
        self.encoder = None
        self.decoder = None
        self.sentiment_predictor = None
        self.autoencoder = None
        
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.latent_rep_size = latent_rep_size

        x = Input(shape=(max_length,))
        self.x_embed = Embedding(vocab_size, 100, input_length=max_length, mask_zero=True)(x)

        vae_loss, encoded = self._build_encoder(self.x_embed, latent_rep_size=latent_rep_size, max_length=max_length)
        self.encoder = Model(x, encoded)

        encoded_input = Input(shape=(latent_rep_size,))
#         predicted_sentiment = self._build_sentiment_predictor(encoded_input)
#         self.sentiment_predictor = Model(encoded_input, predicted_sentiment)

        decoded = self._build_decoder(encoded_input, vocab_size, max_length)
        self.decoder = Model(encoded_input, decoded)

#         self.autoencoder = Model(x, [self._build_decoder(encoded, vocab_size, max_length), self._build_sentiment_predictor(encoded)])
        self.model = Model(x, self._build_decoder(encoded, vocab_size, max_length))

        self.model.compile(optimizer='Adam',
                                 loss=vae_loss)
        
    def _build_encoder(self, x, latent_rep_size=200, max_length=300, epsilon_std=0.01):
#         h = Bidirectional(LSTM(500, return_sequences=True, name='lstm_1'), merge_mode='concat')(x)
#         h = Bidirectional(LSTM(500, return_sequences=False, name='lstm_2'), merge_mode='concat')(h)
#         h = Dense(435, activation='relu', name='dense_1')(h)
        h = LSTM(500, name='lstm_1')(x)


        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev=epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation='linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation='linear')(h)

        def vae_loss(x1, x_decoded_mean):
            x1 = K.flatten(self.x_embed)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.cosine(x1, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))
    
    def _build_decoder(self, encoded, vocab_size, max_length):
        repeated_context = RepeatVector(max_length)(encoded)

#         h = LSTM(500, return_sequences=True, name='dec_lstm_1')(repeated_context)
#         h = LSTM(500, return_sequences=True, name='dec_lstm_2')(h)
        h = LSTM(500, return_sequences=True, name='dec_lstm_1')(repeated_context)
#         h = LSTM(500, return_sequences=True, name='dec_lstm_2')(h)
#         decoded = h
        decoded = Flatten()(TimeDistributed(Dense(100, activation='linear'), name='decoded_mean')(h))
        return decoded
    
    def _build_sentiment_predictor(self, encoded):
        h = Dense(100, activation='linear')(encoded)

        return Dense(1, activation='sigmoid', name='pred')(h)

emb_lstm_vae = EMB_LSTM_VAE()

In [375]:
emb_lstm_vae.model.predict(np.random.randint(50000, size=(10,10))).shape

(10, 10)

In [389]:
for i in range(5):
    emb_lstm_vae.model.fit(q1_qq_click_train, q1_qq_click_train, verbose=2, batch_size=batch_size)
    pred = get_cosine_sim(emb_lstm_vae.encoder.predict(q1_qq_test), emb_lstm_vae.encoder.predict(q2_qq_test))
    print(auc(df_qq.label.values, pred))

Epoch 1/1
 - 165s - loss: -9.7349e+00
0.507015944317
Epoch 1/1
 - 150s - loss: -9.9135e+00
0.533167293024
Epoch 1/1
 - 151s - loss: -9.9363e+00
0.522159710022
Epoch 1/1
 - 152s - loss: -9.9853e+00
0.542862861082
Epoch 1/1
 - 149s - loss: -9.9944e+00
0.524605839578


In [391]:
pred = get_cosine_sim(q1_qq_test, q2_qq_test)
auc(df_qq.label.values, pred)

0.52507282794814203

In [392]:
for i in range(5):
    emb_lstm_vae.model.fit(q2_qq_click_train, q2_qq_click_train, verbose=2, batch_size=batch_size)
    pred = get_cosine_sim(emb_lstm_vae.encoder.predict(q1_qq_test), emb_lstm_vae.encoder.predict(q2_qq_test))
    print(auc(df_qq.label.values, pred))

Epoch 1/1
 - 148s - loss: -9.9773e+00
0.520213925148
Epoch 1/1
 - 149s - loss: -9.9963e+00
0.516400186795
Epoch 1/1
 - 148s - loss: -9.9737e+00
0.530832351175
Epoch 1/1
 - 148s - loss: -9.9949e+00
0.523783050546
Epoch 1/1
 - 149s - loss: -9.9970e+00
0.516822700082
